Before running inference:
1. Run [3.2_Model_conversion_trt.ipynb](3.2_Model_conversion_trt.ipynb) to convert models .onnx files into .trt
2. Make sure TensorRT [binaries](https://developer.nvidia.com/tensorrt/download/10x) are present and added to [Path](https://medium.com/@guillermovc/how-to-install-tensorrt-in-windows-10-71a4033c4407).
3. Make sure pycuda and tensorrt is installed via pip.

In [1]:
import torchvision.transforms as transforms
import cv2

from helpers.processing import display_img_with_map
from helpers.inference_trt import InferenceTensorRT

In [2]:
engine_path = 'efficientnet_v2_s_downscaled_pcb_fp32.trt'
trt_inference = InferenceTensorRT(engine_path)

In [3]:
image = cv2.imread('../docs/sample_data/01_short_04_1926_1070.png', cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
transform = transforms.Compose([transforms.ToTensor()])
tensor = transform(image).unsqueeze(0)

In [ ]:
%%timeit
trt_inference.predict(tensor.numpy())

In [ ]:
outputs = trt_inference.predict(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [ ]:
engine_path = 'efficientnet_v2_s_downscaled_pcb_fp16.trt'
trt_inference = InferenceTensorRT(engine_path)

In [ ]:
%%timeit
trt_inference.infer(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [20]:
engine_path = 'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb_fp32.trt'
trt_inference = InferenceTensorRT(engine_path)

In [ ]:
%%timeit
trt_inference.predict(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [23]:
engine_path = 'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb_fp16.trt'
trt_inference = InferenceTensorRT(engine_path)

In [ ]:
%%timeit
trt_inference.predict(tensor.numpy())

In [ ]:
outputs = trt_inference.predict(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)